# radial_flux_profiles.ipynb

This notebook utilizes user input to 1) select galaxies in a range of stellar masses and redshift, 2) create radial flux profiles in the eight HST bands of the CANDELS survey, 3) prepare an ASCII file that concatenates radial flux profiles of a subset of galaxies, and 4) prepare FITS files of the radial flux profiles using the concatenated flux profile ASCII files.

Radial profiles are generated through an adaptive binning scheme, in which the S/N within an annulus must meet a certain threshold or the annulus will increase in size to meet the S/N threshold. The user indicates the pixel resolution of the radial flux profiles, how do perform outlier rejection, whether interlacing is desired, and other settings. 

HST cutouts in each band must be prepared in advance, and are read in by this notebook. Each galaxy that has cutouts prepared must have a directory in place where the flux profile will be stored. Cutouts must already point spread function (PSF) - matched to the resolution of the F160W band. 

To run the notebook:
1. import modules below; check that versions and functions imported are compatible with what was used to write the notebook
2. user selects list of galaxies for which radial flux profiles will be produced, as well as catalogs of morphological properties of the galaxies
3. user indicates preferences for producing radial flux profiles
4. radial flux profiles are created
5. user assembles radial profiles into larger ASCII files, to be used later to create FITS files
6. user creates FITS files to be fitted using BEAGLE

### Import modules

This notebook was written using the following versions:

| module | version |
|:---   |:---: |
|numpy  | 1.17.2  |
|matplotlib | 2.2.2 |
|scipy | 1.3.1 |
| astropy | 2.0.5 |
| photutils | 0.6 |

In [1]:
# numpy
import numpy as np
from numpy import unravel_index
print( 'Using numpy version %s' % np.__version__ )

# matplotlib and inline display in notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
print( 'Using matplotlib version %s' % mpl.__version__ )

# matplotlib toolkits
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# scipy
import scipy as scp
from scipy.ndimage import label
from scipy.interpolate import interp1d
from scipy.stats import sigmaclip, iqr
print( 'Using scipy version %s' % scp.__version__ )

# system
import glob, os

# astropy
import astropy as astro
from astropy.table import Table
from astropy.io import ascii, fits
from astropy.stats import sigma_clip
import astropy.units as u
from astropy.cosmology import Planck15
from astropy.convolution import convolve_fft
from astropy.coordinates import SkyCoord
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS
print( 'Using astropy version %s' % astro.__version__ )

# photutils
import photutils as phot
from photutils import EllipticalAperture, EllipticalAnnulus, aperture_photometry, CircularAnnulus, SkyEllipticalAnnulus, SkyEllipticalAperture
from photutils.isophote import Ellipse, EllipseGeometry, EllipseSample, EllipseFitter
print( 'Using photutils version %s' % phot.__version__ )

# text below modifies matplotlib settings so LaTeX can be used in plots
params = {'text.usetex': False, 'mathtext.fontset': 'stixsans'}
plt.rcParams.update(params)

Using numpy version 1.21.5
Using matplotlib version 3.5.3
Using scipy version 1.7.3
Using astropy version 4.3.1
Using photutils version 1.1.0


### Define functions

In [2]:
def centroid(im,region=None):
    """Compute centroid of image (M10/M00,M01/M00) (y,x), region: use this region to compute centroid"""
    if not(region is None):
        im=im[region[0]:region[1]] #region[2]:region[3]]
        offset=[region[0],region[2]]
    else: 
        offset=[0,0]
    m00=np.sum(im)
    m01=np.sum(np.sum(im,axis=1)*np.arange(im.shape[0]))
    m10=np.sum(np.sum(im,axis=0)*np.arange(im.shape[1]))
    return [m01/m00+offset[0], m10/m00+offset[1]]

def compute_total_flux_and_error(hdulist, band, field, aper, split_err=False):
    # compute flux
    aper_phot = aperture_photometry( hdulist[0].data, aper )
    cut_flux = aper_phot['aperture_sum'][0]
    
    # make an image of the counts, science image x exposure time image
    cts_img = hdulist[0].data * hdulist[2].data
    
    # invert the weight map to get a variance map
    var_img = np.ones( ( hdulist[0].data.shape[0], hdulist[0].data.shape[0] ) ) / hdulist[1].data
            
    # multiply the variance map by the square of the exposure time image
    var_cts_img = var_img * np.square( hdulist[2].data )
            
    # compute the total variance, the sum of the counts images and the (variance x exptime^2) image
    total_var = cts_img + var_cts_img
            
    # divide the sqrt of total variance by the exptime image to get the uncertainty image
    total_unc = np.sqrt( total_var ) / hdulist[2].data
    
    if split_err:
        # compute Poisson noise of source
        # first compute Poisson variance within aperture, then compute sqrt(var) / exptime within aperture
        source_poisson_err_img = np.sqrt( cts_img ) / hdulist[2].data
        source_poisson_phot = aperture_photometry( np.square( source_poisson_err_img ), aper )
        source_poisson_err = np.sqrt( source_poisson_phot['aperture_sum'][0] )
        
        
        # compute instrumental+background Poisson noise
        # first compute instrumental variance within aperture, then compute sqrt(var) / exptime within aperture
        bckg_poisson_err_img = np.sqrt( var_cts_img ) / hdulist[2].data
        bckg_poisson_phot = aperture_photometry( np.square( bckg_poisson_err_img ), aper )
        bckg_poisson_err = np.sqrt( bckg_poisson_phot['aperture_sum'][0] )
    
    # aperture photometry of all Poisson noise terms
    aper_phot_unc = aperture_photometry( np.square( total_unc ), aper )
    
    # sky subtraction error
    if field == 'goodss':
        if band == '435':
            a, b = 0.42, 1.34
        elif band == '606':
            a, b = 0.36, 1.48
        elif band == '775':
            a, b = 0.39, 1.35
        elif band == '814':
            a, b = 0.32, 1.71
        elif band == '850':
            a, b = 0.35, 1.52
        elif band == '125':
            a, b = 0.48, 1.36
        elif band == '140':
            a, b = 0.52, 1.17
        elif band == '160':
            a, b = 0.50, 1.34
        
    elif field == 'goodsn':
        if band == '435':
            a, b = 0.31, 1.51
        elif band == '606':
            a, b = 0.33, 1.42
        elif band == '775':
            a, b = 0.36, 1.45
        elif band == '850':
            a, b = 0.40, 1.45
        elif band == '125':
            a, b = 0.56, 1.23
        elif band == '140':
            a, b = 0.27, 1.22
        elif band == '160':
            a, b = 0.65, 1.20
            
    # compute the square root of the weight within the aperture
    aper_phot_wht = aperture_photometry( np.sqrt( hdulist[1].data ), aper )
    cut_weight = aper_phot_wht['aperture_sum'][0]
    
    # compute sky subtraction error
    sky_sub_err = a * np.sqrt( aper.area ) ** b / ( cut_weight / aper.area )
    
    cut_err = np.sqrt( aper_phot_unc['aperture_sum'][0] + sky_sub_err ** 2 )
    
    if split_err:
        return cut_flux, cut_err, source_poisson_err, bckg_poisson_err, sky_sub_err
    else:
        return cut_flux, cut_err
    
def snr_limits( flux_f, min_snr=3 ):
    
    snr_lim_435 = snr_limit_one_band( prof_f=flux_f, snr_opt='F435W_SN', snr_limit=min_snr, flux_=True )
    snr_lim_606 = snr_limit_one_band( prof_f=flux_f, snr_opt='F606W_SN', snr_limit=min_snr, flux_=True )
    snr_lim_775 = snr_limit_one_band( prof_f=flux_f, snr_opt='F775W_SN', snr_limit=min_snr, flux_=True )
    snr_lim_850 = snr_limit_one_band( prof_f=flux_f, snr_opt='F850LP_SN', snr_limit=min_snr, flux_=True )
    
    snr_lim_125 = snr_limit_one_band( prof_f=flux_f, snr_opt='F125W_SN', snr_limit=min_snr, flux_=True )
    snr_lim_140 = snr_limit_one_band( prof_f=flux_f, snr_opt='F140W_SN', snr_limit=min_snr, flux_=True )
    snr_lim_160 = snr_limit_one_band( prof_f=flux_f, snr_opt='F160W_SN', snr_limit=min_snr, flux_=True )
    
    return np.array( [ snr_lim_435, snr_lim_606, snr_lim_775, snr_lim_850, snr_lim_125, snr_lim_140, snr_lim_160 ] )

def snr_limit_one_band( prof_f, snr_opt, snr_limit, flux_=False ):
    
    if flux_:
        snr_opt_str = snr_opt.split('_')[0]
        snr_opt_arr = prof_f[snr_opt_str + '_flux'] / prof_f[snr_opt_str + '_fluxerr']
        
        snr_above_lim = np.where( snr_opt_arr >= snr_limit )[0]
    
        snr_near_lim = []
    
        if len(snr_above_lim) == 0:
            snr_lim = 0
        else:
    
            for i in snr_above_lim[1:len(snr_above_lim)]:
                if i + 1 == len( snr_opt_arr ):
                    continue
            
                if snr_opt_arr[i-1] > snr_limit and snr_opt_arr[i+1] < snr_limit:
                    snr_near_lim.append( i )
        
            if len( snr_near_lim ) == 0:
                snr_lim = np.max( snr_above_lim )
            else:
                snr_lim = np.min( snr_near_lim )
        
    else: 
        snr_above_lim = np.where( prof_f[snr_opt] >= snr_limit )[0]
    
        snr_near_lim = []
    
        if len(snr_above_lim) == 0:
            snr_lim = 0
        else:
    
            for i in snr_above_lim[1:len(snr_above_lim)]:
                if i + 1 == len( prof_f[snr_opt] ):
                    continue
            
                if prof_f[snr_opt][i-1] > snr_limit and prof_f[snr_opt][i+1] < snr_limit:
                    snr_near_lim.append( i )
        
            if len( snr_near_lim ) == 0:
                snr_lim = np.max( snr_above_lim )
            else:
                snr_lim = np.min( snr_near_lim )
            
    return snr_lim

### Retrieve catalogs of galaxy IDs and galaxy properties
Below, the user selects a list galaxy IDs to indicate which objects will have radial flux profiles created, as well as catalogs containing the axis ratios and position angles of selected galaxies. 

The user must specify whether the list of galaxies (file selection_f) is a list of galaxy IDs, or a directory comprising subdirectories titled with the IDs of galaxies.

In [3]:
# selection file comprising galaxies to be fitted
selection_f = ascii.read('/Volumes/Vega/CANDELS/sed_assumptions_paper/goods_param_remaining_final_cuts_no_vis_mar23.txt')
#selection_f = ascii.read('/Users/vega/proposals/hst_cycle_29/abundance_match_sample.txt')

#selection_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/goods_sample_param.txt')

input_type = 'list'

# confirm user input_type is reasonable
assert input_type == 'list' or input_type == 'dir', "input_type must either be 'list' or 'dir'"

# catalogs of galaxy properties (position angle, axis ratio)
#candels_3dhst_match_cat = ascii.read('/Users/vega/CANDELS/sample/goodss_candels_3dhst_kodra_vdw_matched_hmag_25_irac_contam_0.5_use_phot_z_0.4_1.5.txt')

#z_cat = ascii.read('/Users/vega/CANDELS/sample/dust_law_comp/goods_hmag_25_z_0.4_1.5_hst_only_calz_no_burst_tauv_exp_min_msa_8.5_param_match_clean.txt')


#print( np.where( ( selection_f['field'] == "goodss" ) & (selection_f['ID'] == 17031) ) )

### Create radial flux profiles
Radial flux profiles are made as follows. For each galaxy in the user-provided list above, a cutout in the F160W band is multiplied by a segmentation map to determine the centroid of the galaxy. This ensures that neighboring galaxies do not contaminate the calculation of the centroid. The pixel coordinates of the centroid are consided the center of the galaxy. The axis ratio and position angle of the galaxy are then obtained from the CANDELS catalog to create an elliptical annulus centered on the galaxy. Within an annulus, the pixel flux distribution in each HST band undergoes outlier rejection, and the medians of the remaining pixel flux and pixel error distributions are recorded. This procedure repeats for each annulus. Annuli are spaced a minimum distance apart set by the user and each annulus must meet a threshold in signal-to-noise. If the threshold is not met, the annulus increases in its outer radius until either the threshold is met or it exceeds a maximum distance set by the user. Annuli reach the maximum distance when the median SNR in the F160W is lower than a value defined by the user.

Once the inner and outer radii of each annulus are determined, the median pixel flux and median pixel flux error are recorded for each HST band, taking into account outlier rejection according to the user's selections. After performing outlier rejection, the median pixel flux and median pixel flux error in each HST band in each annulus is saved in an ASCII file. 

The user must input a number of parameters and boolean variables, and an array mask, to create radial flux profiles. The array mask determines how much of the selection file is run at one time and divvies up the number of galaxies that will have radial flux profiles made in one calculation. This is implemented as photutils passes a warning when creating an elliptical annulus of the center of a galaxy, as the inner radius is set to 0, and for hundreds of galaxies these warnings make the notebook excessively large (the warning doesn't actually affect the resulting central flux, but I cannot seem to remove them). 

The parameters that must be set by the user are:
  - min_pix: the minimum number of pixels within an annulus
  - min_snr: the minimum signal-to-noise within the annulus
  - min_diff: the minimum difference in pixels between the inner and outer radii of the annulus
  - max_diff: the maximum difference in pixels between the inner and outer radii of the annulus

The array mask set by the user is:
  - input_list: a mask, of the range() array of length the selection file, which is used to create radial flux profiles in multiple parts

The boolean variables set by the user are:
  - plot_mask: if True, the pixel mask of each annulus will be plotted
  - interlace: if True, the differences of the inner and outer radii of the annuli will be multiplied by 0.5 and added to the exisitng inner radii of the annuli; this effectively increases the spatial resolution of the radial profile without oversampling the PSF
  - do_sigma_clip: if True, outlier rejection is performed via sigma-clipping; the following parameters must then be set:
    - sigma_level: the number of standard deviations above or below which a pixel flux value must reach to be rejected
    - sigma_iter: the number of iterations of sigma-clipping performed in each annulus, in each band
  - do_percentile: if True, outlier rejection is performed by percentile-clipping; the following parameter ust then be set:
    - percent_level: the percentile below which and above which a pixel flux value within an annulus will be rejected
  - try_all_rejected_pix: if True, all rejected pixels in all HST band will be excluded from calculating the medians in each HST band per annulus
  - record_hist: if True, save in an ASCII file the distribution of pixel fluxes within an annulus

### User input

In [6]:
# minimum number of pixels within annulus
min_pix = 5

# minimum signal-to-noise within annulus
min_snr = 3 # 10

# minimum difference in pixels for an annulus
min_diff = 2

# maximum difference in pixels for an annulus
max_diff = 5

# use the S/N in one band or in multiple bands to determine extent of the profile
use_one_band_snr = False

# write each error term to the ASCII file
write_each_err = False

# check that min_pix > 0
assert min_pix > 0, 'min_pix must be greater than 0'

# check that min_snr > 0
assert min_snr > 0, 'min_snr must be greater than 0'

# check that min_diff > 0 and min_diff < max_diff
assert min_diff > 0, 'min_diff must be greater than 0'

assert min_diff < max_diff, 'min_diff must be less than max_diff'

# a mask, of the range array of length of the selection file, used to split the creation of the radial flux profiles
#
# if the user wanted to produce all radial flux profiles of all galaxies in a given mass and redshift range, 
# then input_list = range( len( sel_f ) )
#
# if the user wanted to split the production of radial flux profiles in two:
# first launch cell below with input_list = range( len( sel_f ) )[int( 0.5 * len( sel_f ) ):]: 
# then launch cell below with input_list = range( len( sel_f ) )[:int( 0.5 * len( sel_f ) )]: 
#input_list = range( len( selection_f ) )[int( 0.5 * len( selection_f ) ):]
input_list = range( len( selection_f ) )

# boolean variable that plots the pixel mask of the annulus
plot_mask = False

# boolean variable to interlace profiles
interlace = False

# boolean variable to report the flux as the mean
# if use_median = False, the mean is measured
use_mean = False

# boolean variable to report the flux as the median
use_median = False

# boolean variable to measure radii using the mean S/N per annulus
use_mean_snr = False

# boolean variable to measure sum of pixel fluxes
use_sum = True

# boolean variable to perform outlier rejection via sigma clipping in each annulus, in each HST band
do_sigma_clip = False

sigma_level = 4.0 # how many standard deviations above or below the mean a pixel flux value must be to be rejected
sigma_iter = 5 # number of iterations of sigma-clippings

# check that sigma_iter is an integer
if do_sigma_clip:
    assert isinstance( sigma_iter, int ), 'sigma_iter must be an integer'

# boolean variable to perform outlier rejection
do_percentile = False

percent_level = 0.5 # percent level below which, and 1 - percent_level above which pixels are rejected

# check that either do_sigma_clip or do_percentile is True, or that both are False
assert ( do_sigma_clip == True and do_percentile == False) or ( do_percentile == True and do_sigma_clip == False ) or ( do_sigma_clip == False and do_percentile == False ), 'Either do_sigma_clip or do_percentile must be True, or both must be False'

# boolean variable to remove all outlier pixels in all HST bands per annulus
try_all_rejected_pix = False

# boolean variable to record distribution of pixel fluxes within an annulus
record_hist = False

In [7]:
fields = ['goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss']
ids = [10458, 10832, 11879, 12330, 13233, 16197, 17050, 17983, 19591, 20052, 20093, 20367, 21006, 23127, 23375, 23485, 24061, 26799, 26883, 2744, 3772, 3891, 4013, 4651, 4911, 5758, 5974, 602, 6298, 6446, 6545, 8771, 9223, 10303, 12028, 12488, 12779, 14306, 1446, 16264, 19462, 20172, 20310, 23246, 24990, 2551, 25716, 3503, 4210, 5223, 6423, 7365, 8352]
zs = [0.9917, 0.4748, 1.489, 0.688, 0.436, 0.409, 1.312, 1.331, 0.8541, 1.372, 0.849, 0.484, 0.8355, 0.9048, 1.0306, 0.6341, 1.236, 0.7465, 1.435, 0.4097, 0.8403, 0.7504, 1.0148, 0.6393, 0.578, 1.3512, 0.6494, 1.427, 1.052, 0.996, 1.3229, 0.4336, 0.4538, 0.9606, 0.732, 0.629, 0.678, 0.731, 1.298, 0.765, 1.166, 0.525, 0.605, 1.2458, 1.174, 1.4181, 1.2333, 1.26, 0.7396, 1.377, 0.991, 0.7341, 0.715]




In [8]:
selection_f = ascii.read('/Volumes/Vega/CANDELS/sed_assumptions_paper/needs_profiles_nov23.txt')
input_type = 'list'

### Creating radial flux profiles

In [9]:
# extension to file name that indicates user input
fname_ext = ''

if interlace:
    fname_ext += '_interlace'

if do_sigma_clip:
    fname_ext += '_sigma_' + str(sigma_level)
elif do_percentile:
    fname_ext += '_percentile_' + str(percent_level)

if use_mean:
    fname_ext += '_mean'
    
if try_all_rejected_pix:
    fname_ext += '_clip_all_pix'
    
if use_sum:
    fname_ext += '_sum'
    
if use_one_band_snr:
    fname_ext += '_f160w_snr_cut'
else:
    fname_ext += '_multiple_band_snr_cut_2_band'
    
if write_each_err:
    fname_ext += '_split_err'
else:
    fname_ext += ''

length = 200

# creating radial flux profiles
#for i in [1112]:#[1372, 979, 1279, 1411, 998, 1066, 1655, 1511, 1529, 945, 1613]:
#for i in range( len( fields ) ):
for i in range( len( selection_f ) ):

    
    if input_type == 'dir':
        if os.path.isdir( i ):
            gal_id = int( os.path.split( i )[-1] )
        else:
            continue
    
    elif input_type == 'list':
        gal_id, field, z = selection_f['ID'][i], selection_f['field'][i], selection_f['z'][i]
        #gal_id, field, z = ids[i], fields[i], zs[i]
        #ind_ = np.where( (selection_f['field'] == field) & (selection_f['ID'] == gal_id) )[0]
    
    """
    if field == 'goodsn':
        z_cat_id = np.where( (z_cat['field'] == 'gdn') & (z_cat['ID'] == gal_id) )[0]
        z = z_cat['z'][z_cat_id[0]]
    elif field == 'goodss':
        z_cat_id = np.where( (z_cat['field'] == 'gds') & (z_cat['ID'] == gal_id) )[0]
        z = z_cat['z'][z_cat_id[0]]
    """
    
    # HST images
    hdulist_160 = fits.open('/Volumes/Vega/CANDELS/candels_images/%s_%d_f160w_images.fits' % ( field, gal_id ) )
    hdu_160 = hdulist_160[0]
    wcs_160 = WCS(hdu_160.header)
    
    length = hdulist_160[0].data.shape[0]
    convert_factor160 = hdu_160.header['PHOT_ZP']
    
    hdulist_140 = fits.open('/Volumes/Vega/CANDELS/candels_images/%s_%d_f140w_images.fits' % ( field, gal_id ) )
    hdu_140 = hdulist_140[0]
    wcs_140 = WCS(hdu_140.header)
    convert_factor140 = hdu_140.header['PHOT_ZP']
    
    hdulist_125 = fits.open('/Volumes/Vega/CANDELS/candels_images/%s_%d_f125w_images.fits' % ( field, gal_id ) )
    hdu_125 = hdulist_125[0]
    wcs_125 = WCS(hdu_125.header)
    convert_factor125 = hdu_125.header['PHOT_ZP']
    
    hdulist_850 = fits.open('/Volumes/Vega/CANDELS/candels_images/%s_%d_f850lp_images.fits' % ( field, gal_id ) )
    hdu_850 = hdulist_850[0]
    wcs_850 = WCS(hdu_850.header)
    convert_factor850 = hdu_850.header['PHOT_ZP']
    
    hdulist_775 = fits.open('/Volumes/Vega/CANDELS/candels_images/%s_%d_f775w_images.fits' % ( field, gal_id ) )
    hdu_775 = hdulist_775[0]
    wcs_775 = WCS(hdu_775.header)
    convert_factor775 = hdu_775.header['PHOT_ZP']
    
    hdulist_606 = fits.open('/Volumes/Vega/CANDELS/candels_images/%s_%d_f606w_images.fits' % ( field, gal_id ) )
    hdu_606 = hdulist_606[0]
    wcs_606 = WCS(hdu_606.header)
    convert_factor606 = hdu_606.header['PHOT_ZP']
    
    hdulist_435 = fits.open('/Volumes/Vega/CANDELS/candels_images/%s_%d_f435w_images.fits' % ( field, gal_id ) )
    hdu_435 = hdulist_435[0]
    wcs_435 = WCS(hdu_435.header)
    convert_factor435 = hdu_435.header['PHOT_ZP']
    
    #hdulist_814 = fits.open('/Volumes/Kassin/CANDELS/candels_images/%s_%d_f814w_images.fits' % ( field, gal_id ) )
    #hdu_814 = hdulist_814[0]
    #wcs_814 = WCS(hdu_814.header)
    #convert_factor814 = hdu_814.header['PHOT_ZP']
    
    # morphological parameters
    coords = SkyCoord(ra=selection_f['RA'][i], dec=selection_f['DEC'][i], unit='deg')
    
    #a_image, b_image, theta_image = candels_3dhst_match_cat['A_IMAGE_1'][cat_id], candels_3dhst_match_cat['B_IMAGE_1'][cat_id], candels_3dhst_match_cat['THETA_IMAGE'][cat_id]
    #ax_ratio = b_image[0] / a_image[0]
    
    ax_ratio, theta_image = selection_f['axis_ratio_SE'][i], selection_f['PA_SE'][i]
    
    theta_rad = theta_image * np.pi / 180. + 0.5 * np.pi
    
    # adaptive annuli binning 
    annuli_inner = []
    annuli_outer = []
    annuli_radii = []
    annuli_areas = []

    # boolean variable used to tell when an annulus has met S/N criterion
    stop = False
            
    # minimum radius of the annulus, min_rad (in pixels), is used to denote the inner radius of
    # each annulus created, and updates after S/N criterion is met
    # maximum radius, max_rad, indicates how far out a profile extends (in pixels)
    min_rad = 0
    max_rad = 100

    while min_rad < max_rad and not stop:
        #print 'New minimum radius', min_rad
                
        for j in range( min_diff, max_diff ):
            next_rad = min_rad + j # update minimum radius
            #print next_rad
                
            if min_rad == 0:
                aper_sky = SkyEllipticalAperture(coords, a=next_rad * u.pixel, b=ax_ratio * next_rad * u.pixel, theta=theta_rad * u.radian)
                aper = aper_sky.to_pixel(wcs=wcs_160)
            else:
                aper_sky = SkyEllipticalAnnulus(coords, a_in=min_rad * u.pixel, a_out=next_rad * u.pixel, b_out=ax_ratio * next_rad * u.pixel, theta=theta_rad * u.radian)
                aper = aper_sky.to_pixel(wcs=wcs_160)
            
            # if only using one band to compute S/N
            if use_one_band_snr:
                # calculate flux and S/N
                cut_flux, cut_err = compute_total_flux_and_error(hdulist=hdulist_160, band='160', field=field, aper=aper)
                
                # if the flux within the annulus is 0, stop immediately
                # this mainly occurs when the profile extends to the edge of the cutout
                # but overall this very rarely happens
                if cut_flux == 0. or cut_err == 0.:
                    stop = True
                    
                aper_snr = cut_flux / cut_err
                
                # if, for some reason, the SNR becomes NaN, stop
                if np.isnan( aper_snr ):
                    stop = True
                    
                # if there are enough pixels in annulus and S/N criterion is met
                if aper_snr >= min_snr:
                    # populate arrays
                    # indicate that a minimum radius of the annulus has been found
                    annuli_inner.append( min_rad )
            
                    annuli_radii.append( np.average( [min_rad, next_rad] ) )
            
                    annuli_areas.append( aper.area )
            
                    min_rad = next_rad # update min radius
                        
                    # indicate that an outer radius of the annulus has been found
                    annuli_outer.append( min_rad )
                    break
                        
                # if S/N criterion is not met, stop
                elif aper_snr < min_snr:
                    stop = True
            
            # if using multiple bands to compute S/N
            else:
                # calculate flux and S/N in all bands
                cut_flux_435, cut_err_435 = compute_total_flux_and_error(hdulist=hdulist_435, band='435', field=field, aper=aper)
                cut_flux_606, cut_err_606 = compute_total_flux_and_error(hdulist=hdulist_606, band='606', field=field, aper=aper)
                cut_flux_775, cut_err_775 = compute_total_flux_and_error(hdulist=hdulist_775, band='775', field=field, aper=aper)
                cut_flux_850, cut_err_850 = compute_total_flux_and_error(hdulist=hdulist_850, band='850', field=field, aper=aper)
                cut_flux_125, cut_err_125 = compute_total_flux_and_error(hdulist=hdulist_125, band='125', field=field, aper=aper)
                cut_flux_140, cut_err_140 = compute_total_flux_and_error(hdulist=hdulist_140, band='140', field=field, aper=aper)
                cut_flux_160, cut_err_160 = compute_total_flux_and_error(hdulist=hdulist_160, band='160', field=field, aper=aper)
                
                if cut_flux_435 == 0. or cut_err_435 == 0.:
                    snr_435 = 0.
                else:
                    snr_435 = cut_flux_435 / cut_err_435
                    
                if cut_flux_606 == 0. or cut_err_606 == 0.:
                    snr_606 = 0.
                else:
                    snr_606 = cut_flux_606 / cut_err_606
                    
                if cut_flux_775 == 0. or cut_err_775 == 0.:
                    snr_775 = 0.
                else:
                    snr_775 = cut_flux_775 / cut_err_775
                    
                if cut_flux_850 == 0. or cut_err_850 == 0.:
                    snr_850 = 0.
                else:
                    snr_850 = cut_flux_850 / cut_err_850
                    
                if cut_flux_125 == 0. or cut_err_125 == 0.:
                    snr_125 = 0.
                else:
                    snr_125 = cut_flux_125 / cut_err_125
                    
                if cut_flux_140 == 0. or cut_err_140 == 0.:
                    snr_140 = 0.
                else:
                    snr_140 = cut_flux_140 / cut_err_140
                    
                if cut_flux_160 == 0. or cut_err_160 == 0.:
                    snr_160 = 0.
                else:
                    snr_160 = cut_flux_160 / cut_err_160
                    
                # array of S/N values
                snrs = np.array([ snr_435, snr_606, snr_775, snr_850, snr_125, snr_140, snr_160 ])
                
                # find how many S/N values meet/exceed min S/N
                num_snr_geq_min = len( np.where( snrs >= min_snr )[0] )
                
                # if there are enough bands with S/N >= min S/N
                if num_snr_geq_min >= 2:
                    # populate arrays
                    # indicate that a minimum radius of the annulus has been found
                    annuli_inner.append( min_rad )
            
                    annuli_radii.append( np.average( [min_rad, next_rad] ) )
            
                    annuli_areas.append( aper.area )
            
                    min_rad = next_rad # update min radius
                        
                    # indicate that an outer radius of the annulus has been found
                    annuli_outer.append( min_rad )
                    break
                        
                # if S/N criterion is not met, stop
                else:
                    stop = True
            
    # saving annuli bins
    annuli_radii = np.asarray( annuli_radii )

    annuli_inner = np.asarray( annuli_inner )
    annuli_outer = np.asarray( annuli_outer )
        
    annuli_areas = np.asarray( annuli_areas )
            
    # interlacing
    if interlace:

        annuli_interlace = 0.5 * (annuli_outer - annuli_inner) + annuli_inner

        annuli_inner_interlace = []
        annuli_outer_interlace = []

        for i in range( len( annuli_interlace ) ):
            # inner
            annuli_inner_interlace.append( annuli_inner[i] )
    
            if i < len( annuli_interlace ) - 1:
                annuli_inner_interlace.append( annuli_interlace[i] )
    
            # outer
            annuli_outer_interlace.append( annuli_outer[i] )
    
            if i < len( annuli_interlace ) - 1:
                annuli_outer_interlace.append( annuli_interlace[i+1] )
        
        annuli_inner_interlace = np.asarray( annuli_inner_interlace )
        annuli_outer_interlace = np.asarray( annuli_outer_interlace )
    
        annuli_inner = np.asarray( annuli_inner_interlace )
        annuli_outer = np.asarray( annuli_outer_interlace )
    
        annuli_radii = 0.5 * (annuli_outer_interlace - annuli_inner_interlace) + annuli_inner_interlace
        
    # creating arrays for the fluxes and flux errors in each HST band
    prof_160_flux = []
    prof_160_err = []

    prof_140_flux = []
    prof_140_err = []

    prof_125_flux = []
    prof_125_err = []

    prof_850_flux = []
    prof_850_err = []

    prof_775_flux = []
    prof_775_err = []

    prof_606_flux = []
    prof_606_err = []

    prof_435_flux = []
    prof_435_err = []
    
    #prof_814_flux = []
    #prof_814_err = []
    
    if write_each_err:
        prof_160_source_err = []
        prof_160_bckg_err = []
        prof_160_sky_err = []
        
        prof_140_source_err = []
        prof_140_bckg_err = []
        prof_140_sky_err = []
        
        prof_125_source_err = []
        prof_125_bckg_err = []
        prof_125_sky_err = []
        
        prof_850_source_err = []
        prof_850_bckg_err = []
        prof_850_sky_err = []
        
        prof_775_source_err = []
        prof_775_bckg_err = []
        prof_775_sky_err = []
        
        prof_606_source_err = []
        prof_606_bckg_err = []
        prof_606_sky_err = []
        
        prof_435_source_err = []
        prof_435_bckg_err = []
        prof_435_sky_err = []

    # calculating fluxes in different bands
    for i in range( len( annuli_radii ) ):
            
        if annuli_inner[i] == 0:
            aper_sky = SkyEllipticalAperture(coords, a=annuli_outer[i] * u.pixel, b=ax_ratio * annuli_outer[i] * u.pixel, theta=theta_rad * u.radian)
            aper = aper_sky.to_pixel(wcs=wcs_160)
        else:
            aper_sky = SkyEllipticalAnnulus(coords, a_in=annuli_inner[i] * u.pixel, a_out=annuli_outer[i] * u.pixel, b_out=ax_ratio * annuli_outer[i] * u.pixel, theta=theta_rad * u.radian)
            aper = aper_sky.to_pixel(wcs=wcs_160)
    
        # F435W
        # calculate flux and S/N
        if write_each_err:
            cut_flux_435, cut_err_435, cut_source_435, cut_bckg_435, cut_sky_435 = compute_total_flux_and_error(hdulist=hdulist_435, band='435', field=field, aper=aper, split_err=True)
        else:
            cut_flux_435, cut_err_435 = compute_total_flux_and_error(hdulist=hdulist_435, band='435', field=field, aper=aper)
                
        # F606W
        # calculate flux and S/N
        if write_each_err:
            cut_flux_606, cut_err_606, cut_source_606, cut_bckg_606, cut_sky_606 = compute_total_flux_and_error(hdulist=hdulist_606, band='606', field=field, aper=aper, split_err=True)
        else:
            cut_flux_606, cut_err_606 = compute_total_flux_and_error(hdulist=hdulist_606, band='606', field=field, aper=aper)
                
        # F775W
        # calculate flux and S/N
        if write_each_err:
            cut_flux_775, cut_err_775, cut_source_775, cut_bckg_775, cut_sky_775 = compute_total_flux_and_error(hdulist=hdulist_775, band='775', field=field, aper=aper, split_err=True)
        else:
            cut_flux_775, cut_err_775 = compute_total_flux_and_error(hdulist=hdulist_775, band='775', field=field, aper=aper)
                
        # F850LP
        # calculate flux and S/N    
        if write_each_err:
            cut_flux_850, cut_err_850, cut_source_850, cut_bckg_850, cut_sky_850 = compute_total_flux_and_error(hdulist=hdulist_850, band='850', field=field, aper=aper, split_err=True)
        else:
            cut_flux_850, cut_err_850 = compute_total_flux_and_error(hdulist=hdulist_850, band='850', field=field, aper=aper)
                
        # F125W
        # calculate flux and S/N
        if write_each_err:
            cut_flux_125, cut_err_125, cut_source_125, cut_bckg_125, cut_sky_125 = compute_total_flux_and_error(hdulist=hdulist_125, band='125', field=field, aper=aper, split_err=True)
        else:
            cut_flux_125, cut_err_125 = compute_total_flux_and_error(hdulist=hdulist_125, band='125', field=field, aper=aper)
                
        # F140W
        # calculate flux and S/N    
        if write_each_err:
            cut_flux_140, cut_err_140, cut_source_140, cut_bckg_140, cut_sky_140 = compute_total_flux_and_error(hdulist=hdulist_140, band='140', field=field, aper=aper, split_err=True)
        else:
            cut_flux_140, cut_err_140 = compute_total_flux_and_error(hdulist=hdulist_140, band='140', field=field, aper=aper)
                
        # F160W
        # calculate flux and S/N    
        if write_each_err:
            cut_flux_160, cut_err_160, cut_source_160, cut_bckg_160, cut_sky_160 = compute_total_flux_and_error(hdulist=hdulist_160, band='160', field=field, aper=aper, split_err=True)
        else:
            cut_flux_160, cut_err_160 = compute_total_flux_and_error(hdulist=hdulist_160, band='160', field=field, aper=aper)
        
        # F814W
        #cut_flux_814, cut_err_814 = compute_total_flux_and_error(hdulist=hdulist_814, band='814', field=field, aper=aper)
        
        # append to arrays
        prof_160_flux.append( cut_flux_160 ) 
        prof_160_err.append( cut_err_160 ) 
        
        prof_140_flux.append( cut_flux_140 ) 
        prof_140_err.append( cut_err_140 ) 
        
        prof_125_flux.append( cut_flux_125 ) 
        prof_125_err.append( cut_err_125 ) 
        
        prof_850_flux.append( cut_flux_850 ) 
        prof_850_err.append( cut_err_850 ) 
        
        prof_775_flux.append( cut_flux_775 ) 
        prof_775_err.append( cut_err_775 ) 
        
        prof_606_flux.append( cut_flux_606 ) 
        prof_606_err.append( cut_err_606 ) 
        
        prof_435_flux.append( cut_flux_435 ) 
        prof_435_err.append( cut_err_435 ) 
        
        #prof_814_flux.append( cut_flux_814 ) 
        #prof_814_err.append( cut_err_814 ) 
        
        if write_each_err:
            prof_160_source_err.append( cut_source_160 )
            prof_160_bckg_err.append( cut_bckg_160 )
            prof_160_sky_err.append( cut_sky_160 )
        
            prof_140_source_err.append( cut_source_140 )
            prof_140_bckg_err.append( cut_bckg_140 )
            prof_140_sky_err.append( cut_sky_140 )
        
            prof_125_source_err.append( cut_source_125 )
            prof_125_bckg_err.append( cut_bckg_125 )
            prof_125_sky_err.append( cut_sky_125 )
        
            prof_850_source_err.append( cut_source_850 )
            prof_850_bckg_err.append( cut_bckg_850 )
            prof_850_sky_err.append( cut_sky_850 )
        
            prof_775_source_err.append( cut_source_775 )
            prof_775_bckg_err.append( cut_bckg_775 )
            prof_775_sky_err.append( cut_sky_775 )
        
            prof_606_source_err.append( cut_source_606 )
            prof_606_bckg_err.append( cut_bckg_606 )
            prof_606_sky_err.append( cut_sky_606 )
        
            prof_435_source_err.append( cut_source_435 )
            prof_435_bckg_err.append( cut_bckg_435 )
            prof_435_sky_err.append( cut_sky_435 )
                    
    # finishing up
    prof_160_flux = np.asarray( prof_160_flux )
    prof_160_err = np.asarray( prof_160_err )

    prof_140_flux = np.asarray( prof_140_flux )
    prof_140_err = np.asarray( prof_140_err )

    prof_125_flux = np.asarray( prof_125_flux )
    prof_125_err = np.asarray( prof_125_err )

    prof_850_flux = np.asarray( prof_850_flux )
    prof_850_err = np.asarray( prof_850_err )

    prof_775_flux = np.asarray( prof_775_flux )
    prof_775_err = np.asarray( prof_775_err )

    prof_606_flux = np.asarray( prof_606_flux )
    prof_606_err = np.asarray( prof_606_err )

    prof_435_flux = np.asarray( prof_435_flux )
    prof_435_err = np.asarray( prof_435_err )
    
    # writing results to file
    fname = '/Volumes/Vega/CANDELS/radial_flux_profiles/%s_%d_adaptive_annuli_bin_size_%d%s_flux_profiles.txt' % (field, gal_id, min_diff, fname_ext)
    #fname = '/Users/vega/CANDELS/sed_assumptions_paper/profiles_extent/%s_%d_adaptive_annuli_bin_size_%d%s_flux_profiles.txt' % (field, gal_id, min_diff, fname_ext)
    #fname = '/Users/vega/proposals/hst_cycle_29/profiles/%s_%d_adaptive_annuli_bin_size_%d%s_flux_profiles.txt' % (field, gal_id, min_diff, fname_ext)
    profile_f = open(fname, 'w')
    
    if write_each_err:
        
        profile_f.write('sma_pix inner_sma_pix outer_sma_pix axis_ratio position_angle_radian z area_pix F435W_flux F435W_fluxerr F435W_source F435W_bckg F435W_sky F606W_flux F606W_fluxerr F606W_source F606W_bckg F606W_sky F775W_flux F775W_fluxerr F775W_source F775W_bckg F775W_sky F850LP_flux F850LP_fluxerr F850LP_source F850LP_bckg F850LP_sky F125W_flux F125W_fluxerr F125W_source F125W_bckg F125W_sky F140W_flux F140W_fluxerr F140W_source F140W_bckg F140W_sky F160W_flux F160W_fluxerr F160W_source F160W_bckg F160W_sky\n')
        profile_f.close()
            
        for j in range( len( annuli_radii ) ):    
            profile_f = open(fname, 'a+')
            profile_f.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16} {17} {18} {19} {20} {21} {22} {23} {24} {25} {26} {27} {28} {29} {30} {31} {32} {33} {34} {35} {36} {37} {38} {39} {40} {41}\n'.format( annuli_radii[j], annuli_inner[j], annuli_outer[j], '%.3f' % ax_ratio, '%.3f' % theta_rad, z, '%.2f' % annuli_areas[j],
                '%.3e' % (prof_435_flux[j] * convert_factor435), '%.3e' % (prof_435_err[j] * convert_factor435), 
                '%.3e' % (prof_435_source_err[j] * convert_factor435), '%.3e' % (prof_435_bckg_err[j] * convert_factor435), '%.3e' % (prof_435_sky_err[j] * convert_factor435), 
                '%.3e' % (prof_606_flux[j] * convert_factor606), '%.3e' % (prof_606_err[j] * convert_factor606),
                '%.3e' % (prof_606_source_err[j] * convert_factor606), '%.3e' % (prof_606_bckg_err[j] * convert_factor606), '%.3e' % (prof_606_sky_err[j] * convert_factor606),
                '%.3e' % (prof_775_flux[j] * convert_factor775), '%.3e' % (prof_775_err[j] * convert_factor775), 
                '%.3e' % (prof_775_source_err[j] * convert_factor775), '%.3e' % (prof_775_bckg_err[j] * convert_factor775), '%.3e' % (prof_775_sky_err[j] * convert_factor775),
                '%.3e' % (prof_850_flux[j] * convert_factor850), '%.3e' % (prof_850_err[j] * convert_factor850), 
                '%.3e' % (prof_850_source_err[j] * convert_factor850), '%.3e' % (prof_850_bckg_err[j] * convert_factor850), '%.3e' % (prof_850_sky_err[j] * convert_factor850),
                '%.3e' % (prof_125_flux[j] * convert_factor125), '%.3e' % (prof_125_err[j] * convert_factor125),
                '%.3e' % (prof_125_source_err[j] * convert_factor125), '%.3e' % (prof_125_bckg_err[j] * convert_factor125), '%.3e' % (prof_125_sky_err[j] * convert_factor125),
                '%.3e' % (prof_140_flux[j] * convert_factor140), '%.3e' % (prof_140_err[j] * convert_factor140), 
                '%.3e' % (prof_140_source_err[j] * convert_factor140), '%.3e' % (prof_140_bckg_err[j] * convert_factor140), '%.3e' % (prof_140_sky_err[j] * convert_factor140),
                '%.3e' % (prof_160_flux[j] * convert_factor160), '%.3e' % (prof_160_err[j] * convert_factor160),
                '%.3e' % (prof_160_source_err[j] * convert_factor160), '%.3e' % (prof_160_bckg_err[j] * convert_factor160), '%.3e' % (prof_160_sky_err[j] * convert_factor160) ) )
                #'%.3e' % (prof_814_flux[j] * convert_factor814), '%.3e' % (prof_814_err[j] * convert_factor814) ) )
            profile_f.close()
    else:
            
        profile_f.write('sma_pix inner_sma_pix outer_sma_pix axis_ratio position_angle_radian z area_pix F435W_flux F435W_fluxerr F606W_flux F606W_fluxerr F775W_flux F775W_fluxerr F850LP_flux F850LP_fluxerr F125W_flux F125W_fluxerr F140W_flux F140W_fluxerr F160W_flux F160W_fluxerr\n')
        profile_f.close()
            
        for j in range( len( annuli_radii ) ):    
            profile_f = open(fname, 'a+')
            profile_f.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16} {17} {18} {19} {20}\n'.format( annuli_radii[j], annuli_inner[j], annuli_outer[j], '%.3f' % ax_ratio, '%.3f' % theta_rad, z, '%.2f' % annuli_areas[j],
                '%.3e' % (prof_435_flux[j] * convert_factor435), '%.3e' % (prof_435_err[j] * convert_factor435), 
                '%.3e' % (prof_606_flux[j] * convert_factor606), '%.3e' % (prof_606_err[j] * convert_factor606),
                '%.3e' % (prof_775_flux[j] * convert_factor775), '%.3e' % (prof_775_err[j] * convert_factor775), 
                '%.3e' % (prof_850_flux[j] * convert_factor850), '%.3e' % (prof_850_err[j] * convert_factor850), 
                '%.3e' % (prof_125_flux[j] * convert_factor125), '%.3e' % (prof_125_err[j] * convert_factor125),
                '%.3e' % (prof_140_flux[j] * convert_factor140), '%.3e' % (prof_140_err[j] * convert_factor140), 
                '%.3e' % (prof_160_flux[j] * convert_factor160), '%.3e' % (prof_160_err[j] * convert_factor160) ) )
                #'%.3e' % (prof_814_flux[j] * convert_factor814), '%.3e' % (prof_814_err[j] * convert_factor814) ) )
            profile_f.close()
        
    hdulist_160.close()
    hdulist_140.close()
    hdulist_125.close()
    hdulist_850.close()
    hdulist_775.close()
    hdulist_606.close()
    hdulist_435.close()
    
    #hdulist_814.close()
                

/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in double_scalars
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/env

/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in double_scalars
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/env

/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in double_scalars
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/env

/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in double_scalars
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in sqrt
/opt/anaconda3/env

### Create radial profiles of 3DHST images

In [8]:
# extension to file name that indicates user input
fname_ext = ''

if interlace:
    fname_ext += '_interlace'

if do_sigma_clip:
    fname_ext += '_sigma_' + str(sigma_level)
elif do_percentile:
    fname_ext += '_percentile_' + str(percent_level)

if use_mean:
    fname_ext += '_mean'
    
if try_all_rejected_pix:
    fname_ext += '_clip_all_pix'
    
if use_sum:
    fname_ext += '_sum'
    
# creating radial flux profiles
for i in [1112]:
    
    if input_type == 'dir':
        if os.path.isdir( i ):
            gal_id = int( os.path.split( i )[-1] )
        else:
            continue
    
    elif input_type == 'list':
        gal_id, field, z = selection_f['ID'][i], selection_f['field'][i], selection_f['z'][i]
    
    """
    if field == 'goodsn':
        z_cat_id = np.where( (z_cat['field'] == 'gdn') & (z_cat['ID'] == gal_id) )[0]
        z = z_cat['z'][z_cat_id[0]]
    elif field == 'goodss':
        z_cat_id = np.where( (z_cat['field'] == 'gds') & (z_cat['ID'] == gal_id) )[0]
        z = z_cat['z'][z_cat_id[0]]
    """
    
    # HST images
    hdulist_160 = fits.open('/Volumes/Kassin/CANDELS/3dhst_images/%s_%d_f160w_images.fits' % (field, gal_id))
    hdu_160 = hdulist_160[0]
    wcs_160 = WCS(hdu_160.header)
    
    length = hdulist_160[0].data.shape[0]
    convert_factor160 = hdu_160.header['PHOT_ZP']
    
    hdulist_140 = fits.open('/Volumes/Kassin/CANDELS/3dhst_images/%s_%d_f140w_images.fits' % (field, gal_id))
    hdu_140 = hdulist_140[0]
    wcs_140 = WCS(hdu_140.header)
    convert_factor140 = hdu_140.header['PHOT_ZP']
    
    hdulist_125 = fits.open('/Volumes/Kassin/CANDELS/3dhst_images/%s_%d_f125w_images.fits' % (field, gal_id))
    hdu_125 = hdulist_125[0]
    wcs_125 = WCS(hdu_125.header)
    convert_factor125 = hdu_125.header['PHOT_ZP']
    
    hdulist_850 = fits.open('/Volumes/Kassin/CANDELS/3dhst_images/%s_%d_f850lp_images.fits' % (field, gal_id))
    hdu_850 = hdulist_850[0]
    wcs_850 = WCS(hdu_850.header)
    convert_factor850 = hdu_850.header['PHOT_ZP']
    
    hdulist_775 = fits.open('/Volumes/Kassin/CANDELS/3dhst_images/%s_%d_f775w_images.fits' % (field, gal_id))
    hdu_775 = hdulist_775[0]
    wcs_775 = WCS(hdu_775.header)
    convert_factor775 = hdu_775.header['PHOT_ZP']
    
    hdulist_606 = fits.open('/Volumes/Kassin/CANDELS/3dhst_images/%s_%d_f606w_images.fits' % (field, gal_id))
    hdu_606 = hdulist_606[0]
    wcs_606 = WCS(hdu_606.header)
    convert_factor606 = hdu_606.header['PHOT_ZP']
    
    hdulist_435 = fits.open('/Volumes/Kassin/CANDELS/3dhst_images/%s_%d_f435w_images.fits' % (field, gal_id))
    hdu_435 = hdulist_435[0]
    wcs_435 = WCS(hdu_435.header)
    convert_factor435 = hdu_435.header['PHOT_ZP']
    
    # morphological parameters
    coords = SkyCoord(ra=selection_f['RA'][i], dec=selection_f['DEC'][i], unit='deg')
    
    #a_image, b_image, theta_image = candels_3dhst_match_cat['A_IMAGE_1'][cat_id], candels_3dhst_match_cat['B_IMAGE_1'][cat_id], candels_3dhst_match_cat['THETA_IMAGE'][cat_id]
    #ax_ratio = b_image[0] / a_image[0]
    
    ax_ratio, theta_image = selection_f['axis_ratio'][i], selection_f['PA'][i]
    
    theta_rad = theta_image * np.pi / 180. + 0.5 * np.pi
    
    # adaptive annuli binning 
    annuli_inner = []
    annuli_outer = []
    annuli_radii = []
    annuli_areas = []

    # boolean variable used to tell when an annulus has met S/N criterion
    stop = True
            
    # minimum radius of the annulus, min_rad (in pixels), is used to denote the inner radius of
    # each annulus created, and updates after S/N criterion is met
    # maximum radius, max_rad, indicates how far out a profile extends (in pixels)
    min_rad = 0
    max_rad = 100

    while min_rad < max_rad and stop:
        #print 'New minimum radius', min_rad
                
        for j in range( min_diff, max_diff ):
            next_rad = min_rad + j # update minimum radius
            #print next_rad
                
            if min_rad == 0:
                aper_sky = SkyEllipticalAperture(coords, a=next_rad * u.pixel, b=ax_ratio * next_rad * u.pixel, theta=theta_rad * u.radian)
                aper = aper_sky.to_pixel(wcs=wcs_160)
            else:
                aper_sky = SkyEllipticalAnnulus(coords, a_in=min_rad * u.pixel, a_out=next_rad * u.pixel, b_out=ax_ratio * next_rad * u.pixel, theta=theta_rad * u.radian)
                aper = aper_sky.to_pixel(wcs=wcs_160)
        
            # calculate flux and S/N
            aper_phot = aperture_photometry( hdulist_160[0].data, aper )
            
            cut_flux = aper_phot['aperture_sum'][0]
            
            # invert the weight map to get a variance map
            var_img = np.ones( (length, length) ) / hdulist_160[1].data
            
            aper_phot_unc = aperture_photometry( var_img, aper )
    
            cut_err = np.sqrt( aper_phot_unc['aperture_sum'][0] )
            
            # if the flux within the annulus is 0, stop immediately
            # this mainly occurs when the profile extends to the edge of the cutout
            # but overall this very rarely happens
            if cut_flux == 0. or cut_err == 0.:
                stop = False
        
            aper_snr = cut_flux / cut_err #np.nanmedian( cut_flux ) / np.nanmedian( cut_err )
            #print(gal_id, aper_snr)
            
            # if, for some reason, the SNR becomes NaN, stop
            if np.isnan( aper_snr ):
                stop = False
            
            # if there are enough pixels in annulus and S/N criterion is met
            if aper_snr >= min_snr:
                # populate arrays
                # indicate that a minimum radius of the annulus has been found
                annuli_inner.append( min_rad )
            
                annuli_radii.append( np.average( [min_rad, next_rad] ) )
            
                annuli_areas.append( aper.area )
            
                min_rad = next_rad # update min radius
                        
                # indicate that an outer radius of the annulus has been found
                annuli_outer.append( min_rad )
                break
                        
            # if S/N criterion is not met, stop
            elif aper_snr < min_snr:
                stop = False
            
    # saving annuli bins
    annuli_radii = np.asarray( annuli_radii )

    annuli_inner = np.asarray( annuli_inner )
    annuli_outer = np.asarray( annuli_outer )
        
    annuli_areas = np.asarray( annuli_areas )
            
    # interlacing
    if interlace:

        annuli_interlace = 0.5 * (annuli_outer - annuli_inner) + annuli_inner

        annuli_inner_interlace = []
        annuli_outer_interlace = []

        for i in range( len( annuli_interlace ) ):
            # inner
            annuli_inner_interlace.append( annuli_inner[i] )
    
            if i < len( annuli_interlace ) - 1:
                annuli_inner_interlace.append( annuli_interlace[i] )
    
            # outer
            annuli_outer_interlace.append( annuli_outer[i] )
    
            if i < len( annuli_interlace ) - 1:
                annuli_outer_interlace.append( annuli_interlace[i+1] )
        
        annuli_inner_interlace = np.asarray( annuli_inner_interlace )
        annuli_outer_interlace = np.asarray( annuli_outer_interlace )
    
        annuli_inner = np.asarray( annuli_inner_interlace )
        annuli_outer = np.asarray( annuli_outer_interlace )
    
        annuli_radii = 0.5 * (annuli_outer_interlace - annuli_inner_interlace) + annuli_inner_interlace
        
    # creating arrays for the fluxes and flux errors in each HST band
    prof_160_flux = []
    prof_160_err = []

    prof_140_flux = []
    prof_140_err = []

    prof_125_flux = []
    prof_125_err = []

    prof_850_flux = []
    prof_850_err = []

    prof_775_flux = []
    prof_775_err = []

    prof_606_flux = []
    prof_606_err = []

    prof_435_flux = []
    prof_435_err = []
    
    prof_435_weight = []
    prof_606_weight = []
    prof_775_weight = []
    prof_850_weight = []
    prof_125_weight = []
    prof_140_weight = []
    prof_160_weight = []

    # calculating fluxes in different bands
    for i in range( len( annuli_radii ) ):
            
        if annuli_inner[i] == 0:
            aper_sky = SkyEllipticalAperture(coords, a=annuli_outer[i] * u.pixel, b=ax_ratio * annuli_outer[i] * u.pixel, theta=theta_rad * u.radian)
            aper = aper_sky.to_pixel(wcs=wcs_160)
        else:
            aper_sky = SkyEllipticalAnnulus(coords, a_in=annuli_inner[i] * u.pixel, a_out=annuli_outer[i] * u.pixel, b_out=ax_ratio * annuli_outer[i] * u.pixel, theta=theta_rad * u.radian)
            aper = aper_sky.to_pixel(wcs=wcs_160)
    
        # F160W
        # calculate flux and S/N
        aper_phot = aperture_photometry( hdulist_160[0].data, aper )
            
        cut_flux_160 = aper_phot['aperture_sum'][0]
            
        # invert the weight map to get a variance map
        var_img = np.ones( (length, length) ) / hdulist_160[1].data
            
        aper_phot_unc = aperture_photometry( var_img, aper )
    
        cut_err_160 = np.sqrt( aper_phot_unc['aperture_sum'][0] )
        
        aper_phot_wht = aperture_photometry( np.sqrt( hdulist_160[1].data ), aper )
        cut_weight_160 = aper_phot_wht['aperture_sum'][0]
    
        # F140W
        # calculate flux and S/N
        aper_phot = aperture_photometry( hdulist_140[0].data, aper )
            
        cut_flux_140 = aper_phot['aperture_sum'][0]
            
        # invert the weight map to get a variance map
        var_img = np.ones( (length, length) ) / hdulist_140[1].data
            
        aper_phot_unc = aperture_photometry( var_img, aper )
    
        cut_err_140 = np.sqrt( aper_phot_unc['aperture_sum'][0] )
        
        aper_phot_wht = aperture_photometry( np.sqrt( hdulist_140[1].data ), aper )
        cut_weight_140 = aper_phot_wht['aperture_sum'][0]
            
        # F125W
        # calculate flux and S/N
        aper_phot = aperture_photometry( hdulist_125[0].data, aper )
            
        cut_flux_125 = aper_phot['aperture_sum'][0]
            
        # invert the weight map to get a variance map
        var_img = np.ones( (length, length) ) / hdulist_125[1].data
            
        aper_phot_unc = aperture_photometry( var_img, aper )
    
        cut_err_125 = np.sqrt( aper_phot_unc['aperture_sum'][0] )
        
        aper_phot_wht = aperture_photometry( np.sqrt( hdulist_125[1].data ), aper )
        cut_weight_125 = aper_phot_wht['aperture_sum'][0]
    
        # F850LP
        # calculate flux and S/N
        aper_phot = aperture_photometry( hdulist_850[0].data, aper )
            
        cut_flux_850 = aper_phot['aperture_sum'][0]
            
        # invert the weight map to get a variance map
        var_img = np.ones( (length, length) ) / hdulist_850[1].data
            
        aper_phot_unc = aperture_photometry( var_img, aper )
    
        cut_err_850 = np.sqrt( aper_phot_unc['aperture_sum'][0] )
        
        aper_phot_wht = aperture_photometry( np.sqrt( hdulist_850[1].data ), aper )
        cut_weight_850 = aper_phot_wht['aperture_sum'][0]
        
        # F775W
        # calculate flux and S/N
        aper_phot = aperture_photometry( hdulist_775[0].data, aper )
            
        cut_flux_775 = aper_phot['aperture_sum'][0]
            
        # invert the weight map to get a variance map
        var_img = np.ones( (length, length) ) / hdulist_775[1].data
            
        aper_phot_unc = aperture_photometry( var_img, aper )
    
        cut_err_775 = np.sqrt( aper_phot_unc['aperture_sum'][0] )
        
        aper_phot_wht = aperture_photometry( np.sqrt( hdulist_775[1].data ), aper )
        cut_weight_775 = aper_phot_wht['aperture_sum'][0]
    
        # F606W
        # calculate flux and S/N
        aper_phot = aperture_photometry( hdulist_606[0].data, aper )
            
        cut_flux_606 = aper_phot['aperture_sum'][0]
            
        # invert the weight map to get a variance map
        var_img = np.ones( (length, length) ) / hdulist_606[1].data
            
        aper_phot_unc = aperture_photometry( var_img, aper )
    
        cut_err_606 = np.sqrt( aper_phot_unc['aperture_sum'][0] )
        
        aper_phot_wht = aperture_photometry( np.sqrt( hdulist_606[1].data ), aper )
        cut_weight_606 = aper_phot_wht['aperture_sum'][0]
    
        # F435W
        # calculate flux and S/N
        aper_phot = aperture_photometry( hdulist_435[0].data, aper )
            
        cut_flux_435 = aper_phot['aperture_sum'][0]
            
        # invert the weight map to get a variance map
        var_img = np.ones( (length, length) ) / hdulist_435[1].data
            
        aper_phot_unc = aperture_photometry( var_img, aper )
    
        cut_err_435 = np.sqrt( aper_phot_unc['aperture_sum'][0] )
        
        aper_phot_wht = aperture_photometry( np.sqrt( hdulist_435[1].data ), aper )
        cut_weight_435 = aper_phot_wht['aperture_sum'][0]
        
        # append to arrays
        prof_160_flux.append( cut_flux_160 ) 
        prof_160_err.append( cut_err_160 ) 
        
        prof_140_flux.append( cut_flux_140 ) 
        prof_140_err.append( cut_err_140 ) 
        
        prof_125_flux.append( cut_flux_125 ) 
        prof_125_err.append( cut_err_125 ) 
        
        prof_850_flux.append( cut_flux_850 ) 
        prof_850_err.append( cut_err_850 ) 
        
        prof_775_flux.append( cut_flux_775 ) 
        prof_775_err.append( cut_err_775 ) 
        
        prof_606_flux.append( cut_flux_606 ) 
        prof_606_err.append( cut_err_606 ) 
        
        prof_435_flux.append( cut_flux_435 ) 
        prof_435_err.append( cut_err_435 ) 
        
        prof_435_weight.append( cut_weight_435 )
        prof_606_weight.append( cut_weight_606 )
        prof_775_weight.append( cut_weight_775 )
        prof_850_weight.append( cut_weight_850 )
        prof_125_weight.append( cut_weight_125 )
        prof_140_weight.append( cut_weight_140 )
        prof_160_weight.append( cut_weight_160 )
                    
        
    # finishing up
    prof_160_flux = np.asarray( prof_160_flux )
    prof_160_err = np.asarray( prof_160_err )

    prof_140_flux = np.asarray( prof_140_flux )
    prof_140_err = np.asarray( prof_140_err )

    prof_125_flux = np.asarray( prof_125_flux )
    prof_125_err = np.asarray( prof_125_err )

    prof_850_flux = np.asarray( prof_850_flux )
    prof_850_err = np.asarray( prof_850_err )

    prof_775_flux = np.asarray( prof_775_flux )
    prof_775_err = np.asarray( prof_775_err )

    prof_606_flux = np.asarray( prof_606_flux )
    prof_606_err = np.asarray( prof_606_err )

    prof_435_flux = np.asarray( prof_435_flux )
    prof_435_err = np.asarray( prof_435_err )
    
    # writing results to file
    #fname = '/Volumes/Kassin/CANDELS/radial_flux_profiles/%s_%d_adaptive_annuli_bin_size_%d%s_flux_profiles.txt' % (field, gal_id, min_diff, fname_ext)
    fname = '/Volumes/Kassin/CANDELS/3dhst_profiles/%s_%d_adaptive_annuli_bin_size_%d%s_flux_profiles.txt' % (field, gal_id, min_diff, fname_ext)
    profile_f = open(fname, 'w')
            
    profile_f.write('sma_pix inner_sma_pix outer_sma_pix axis_ratio position_angle_radian z area_pix F435W_flux F435W_fluxerr F606W_flux F606W_fluxerr F775W_flux F775W_fluxerr F850LP_flux F850LP_fluxerr F125W_flux F125W_fluxerr F140W_flux F140W_fluxerr F160W_flux F160W_fluxerr F435W_sqrt_weight F606W_sqrt_weight F775W_sqrt_weight F850LP_sqrt_weight F125W_sqrt_weight F140W_sqrt_weight F160W_sqrt_weight\n')
    profile_f.close()
            
    for j in range( len( annuli_radii ) ):    
        profile_f = open(fname, 'a+')
        profile_f.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16} {17} {18} {19} {20} {21} {22} {23} {24} {25} {26} {27}\n'.format( annuli_radii[j], annuli_inner[j], annuli_outer[j], '%.3f' % ax_ratio, '%.3f' % theta_rad, z, '%.2f' % annuli_areas[j],
            '%.3e' % (prof_435_flux[j] * convert_factor435), '%.3e' % (prof_435_err[j] * convert_factor435), 
            '%.3e' % (prof_606_flux[j] * convert_factor606), '%.3e' % (prof_606_err[j] * convert_factor606),
            '%.3e' % (prof_775_flux[j] * convert_factor775), '%.3e' % (prof_775_err[j] * convert_factor775), 
            '%.3e' % (prof_850_flux[j] * convert_factor850), '%.3e' % (prof_850_err[j] * convert_factor850), 
            '%.3e' % (prof_125_flux[j] * convert_factor125), '%.3e' % (prof_125_err[j] * convert_factor125),
            '%.3e' % (prof_140_flux[j] * convert_factor140), '%.3e' % (prof_140_err[j] * convert_factor140), 
            '%.3e' % (prof_160_flux[j] * convert_factor160), '%.3e' % (prof_160_err[j] * convert_factor160),
            '%.2f' % (prof_435_weight[j] / annuli_areas[j]), '%.2f' % (prof_606_weight[j] / annuli_areas[j]),
            '%.2f' % (prof_775_weight[j] / annuli_areas[j]),
            '%.2f' % (prof_850_weight[j] / annuli_areas[j]), '%.2f' % (prof_125_weight[j] / annuli_areas[j]),                                                                   
            '%.2f' % (prof_140_weight[j] / annuli_areas[j]), '%.2f' % (prof_160_weight[j] / annuli_areas[j]) ) )
        profile_f.close()
        
    hdulist_160.close()
    hdulist_140.close()
    hdulist_125.close()
    hdulist_850.close()
    hdulist_775.close()
    hdulist_606.close()
    hdulist_435.close()
                

### Concatenate ASCII files of radial flux profiles
Before preparing FITS files that will be used in SED fitting, the user must concatenate radial flux profiles of different galaxies to a common ASCII file containing multiple flux profiles of galaxies. This is done to ease preparation of the FITS files. 

It is recommended, if galaxies will be fitted within a certain mass and redshift range, to split the concatenated flux profile file into two parts. This will ease the effort spent launching the fits on a cluster. The user can specify how to bifurcate (or otherwise split) the full concatenated flux profile file by modifying the todo_list variable. 

The user must specify the name of the concatenated flux profile file by altering the flux_fname variable. 

### User input

In [7]:
# range in stellar mass
mass1, mass2 = '10.0', '10.4'

# range in redshift
z1, z2 = '1.0', '1.5'

# file of selected galaxies, below referred to as "selection file"
#selection_f = ascii.read('/Users/vega/CANDELS/sample/goods_hmag_25_sample_mass_' + mass1 + '_' + mass2 + '_z_' + z1 + '_' + z2 + '.txt')
selection_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/final_sample_id_list.txt')
#selection_f = ascii.read('/Users/vega/proposals/hst_cycle_29/abundance_match_sample.txt')

# effective binning size in pixels of the profiles
bin_size = 2

# array of indices of the selection file that allows the user to split the full concatenated file
#
# if the user wanted all galaxies in a given mass and redshift range to be included in the concatenated file, 
# then todo_list = range( len( sel_f ) )
#
# if the user wanted to split the full concatenated file in two:
# first launch cell below with todo_list = range( len( sel_f ) )[int( 0.5 * len( sel_f ) ):]: 
# then launch cell below with todo_list = range( len( sel_f ) )[:int( 0.5 * len( sel_f ) )]: 
todo_list = [953, 973, 1040, 1044, 1054, 1097, 1126, 1136, 1182, 1460, 1467, 994, 1033, 1035, 1305, 1379, 1675]#range( len( selection_f ) )#range( len( selection_f ) )[:int( 0.5 * len( selection_f ) )]: 

# name of the flux profile file
#flux_fname = '/Users/vega/CANDELS/flux_files/goods_hmag_25_mass_' + mass1 + '_' + mass2 + '_z_' + z1 + '_' + z2 + '_bin_' + str(bin_size) + '_part1.txt'
#flux_fname = '/Users/vega/proposals/hst_cycle_29/abundance_match_sample_fluxes.txt'
flux_fname = '/Users/vega/CANDELS/sed_assumptions_paper/profiles_extent/profiles_extent_example2_fluxes.txt'



### Create concatenated radial flux profile ASCII file

In [10]:
selection_f = ascii.read('/Volumes/Vega/CANDELS/sed_assumptions_paper/needs_profiles_nov23_fit.txt')

vis_class_f = ascii.read('/Volumes/Vega/CANDELS/sed_assumptions_paper/visual_classifications/delavega_visual_classifications_updated.txt', delimiter='\t', names=['field', 'ID', 'category', 'bands_to_exclude', 'comments'])
non_pristine_f = ascii.read('/Volumes/Vega/CANDELS/sed_assumptions_paper/sample_lists/non_pristine_rad_limit_subset_no_error_issues_nov23.txt', delimiter='\t', names=['field_3', 'ID_3', 'band', 'category'])

flux_fname = '/Volumes/Vega/CANDELS/sed_assumptions_paper/flux_files/remaining_final_cuts_no_vis_fluxes_nov23.txt'

bin_size = 2

from scipy.signal import find_peaks, argrelextrema


In [35]:
from scipy.signal import find_peaks, argrelextrema

selection_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/need_to_be_fitted.txt')

sed_fitting_catalog = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/sed_fitting_catalogs/goods_sample_param_z_dependent_size_cut_good_photo_z_all_assumptions_visual_classifications.txt')

flux_fname = '/Users/vega/CANDELS/sed_assumptions_paper/flux_files/remaining_galaxies_error_issues_cat2_fluxes.txt'

bin_size = 2



In [11]:
# instantiating flux profile file
flux_sel_f = open(flux_fname, 'w')
flux_sel_f.write( 'ID field z F435W_flux F435W_fluxerr F606W_flux F606W_fluxerr F775W_flux F775W_fluxerr F850LP_flux F850LP_fluxerr F125W_flux F125W_fluxerr F140W_flux F140W_fluxerr F160W_flux F160W_fluxerr\n' )
flux_sel_f.close()

for i in range( len( selection_f ) ):
    gal_id, field = selection_f['ID'][i], selection_f['field'][i]
        
    try:
        #flux_f = ascii.read('/Users/vega/proposals/hst_cycle_29/profiles/%s_%d_adaptive_annuli_bin_size_%d_sum_multiple_band_snr_cut_2_band_flux_profiles.txt' % (field, gal_id, bin_size) ) #+ '_sigma_4.0_mean_clip_all_pix_flux_profiles.txt')
        #flux_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/profiles_extent/%s_%d_adaptive_annuli_bin_size_%d_sum_multiple_band_snr_cut_2_band_flux_profiles.txt' % (field, gal_id, bin_size) ) #+ '_sigma_4.0_mean_clip_all_pix_flux_profiles.txt')
        flux_f = ascii.read('/Volumes/Vega/CANDELS/radial_flux_profiles/%s_%d_adaptive_annuli_bin_size_%d_sum_multiple_band_snr_cut_2_band_flux_profiles.txt' % (field, gal_id, bin_size) )
    except:
        print( 'Radial flux profiles have not been produced for %s %d' % (field, gal_id) )
        continue
        
    vis_class_ind = np.where( ( vis_class_f['field'] == field ) & ( vis_class_f['ID'] == gal_id ) )[0]
    category = vis_class_f['category'][vis_class_ind][0] # visual category of galaxy
    
    #cat_ind = np.where( ( sed_fitting_catalog['field'] == field ) & ( sed_fitting_catalog['ID'] == gal_id ) )[0]
    #category = sed_fitting_catalog['category'][cat_ind][0]
    #band = sed_fitting_catalog['band'][cat_ind][0]
        
    z = flux_f['z'][0]
    
    if category == 1:
        continue
    
    elif category >= 3:
        # compute radius limits
        snr_lims_5 = snr_limits( flux_f, min_snr=5. )
    
        # using one WFC3 band to determine the radius limit as a function of redshift
        if z >= 0.8: 
            snr_lim_5 = snr_lims_5[6] # F160W
        else:
            snr_lim_5 = snr_lims_5[4] # F125W
    
    elif category == 2:
        # using SB profile to determine radius limit using WFC3 bands
        non_pristine_ind = np.where( (non_pristine_f['ID_3'] == gal_id) & (non_pristine_f['field_3'] == field) )[0]
        band = non_pristine_f['band'][non_pristine_ind][0]
        #print(band)

        sb_profile = -2.5 * np.log10( flux_f['%s_flux' % band] / ( flux_f['area_pix'] * 0.06 ** 2) ) + 8.9
    
        # find relative maxima 
        rel_max = argrelextrema( sb_profile, np.greater )
        
        snr_lim_5 = rel_max[0][0]
        
    print( snr_lim_5 )
    
    # accounting for errors in bandpasses
    #band_to_exclude = sed_fitting_catalog['bands_to_exclude'][cat_ind][0].replace(',', '')
    band_to_exclude = vis_class_f['bands_to_exclude'][vis_class_ind]
    
    print(gal_id, category, band_to_exclude)
    
    # if the data in F850LP are affected
    if band_to_exclude == 'F850LP':
        
        for j in range( snr_lim_5 + 1 ):
        
            flux_sel_f = open(flux_fname, 'a+')
            flux_sel_f.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16}\n'.format(
                field + '_' + str(gal_id) + '_' + str(j), field, flux_f['z'][j], 
                flux_f['F435W_flux'][j], flux_f['F435W_fluxerr'][j], 
                flux_f['F606W_flux'][j], flux_f['F606W_fluxerr'][j],
                flux_f['F775W_flux'][j], flux_f['F775W_fluxerr'][j], 
                -99., -99.,
                flux_f['F125W_flux'][j], flux_f['F125W_fluxerr'][j], 
                flux_f['F140W_flux'][j], flux_f['F140W_fluxerr'][j], 
                flux_f['F160W_flux'][j], flux_f['F160W_fluxerr'][j] ))
            flux_sel_f.close()
    
    # if the data in F140W are affected
    elif band_to_exclude == 'F140W':
        
        for j in range( snr_lim_5 + 1 ):
        
            flux_sel_f = open(flux_fname, 'a+')
            flux_sel_f.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16}\n'.format(
                field + '_' + str(gal_id) + '_' + str(j), field, flux_f['z'][j], 
                flux_f['F435W_flux'][j], flux_f['F435W_fluxerr'][j], 
                flux_f['F606W_flux'][j], flux_f['F606W_fluxerr'][j],
                flux_f['F775W_flux'][j], flux_f['F775W_fluxerr'][j], 
                flux_f['F850LP_flux'][j], flux_f['F850LP_fluxerr'][j],
                flux_f['F125W_flux'][j], flux_f['F125W_fluxerr'][j], 
                -99., -99., 
                flux_f['F160W_flux'][j], flux_f['F160W_fluxerr'][j] ))
            flux_sel_f.close()
    
    # if no bands are affected
    else:    
        for j in range( snr_lim_5 + 1 ):
        
            flux_sel_f = open(flux_fname, 'a+')
            flux_sel_f.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16}\n'.format(
                field + '_' + str(gal_id) + '_' + str(j), field, flux_f['z'][j], 
                flux_f['F435W_flux'][j], flux_f['F435W_fluxerr'][j], 
                flux_f['F606W_flux'][j], flux_f['F606W_fluxerr'][j],
                flux_f['F775W_flux'][j], flux_f['F775W_fluxerr'][j], 
                flux_f['F850LP_flux'][j], flux_f['F850LP_fluxerr'][j],
                flux_f['F125W_flux'][j], flux_f['F125W_fluxerr'][j], 
                flux_f['F140W_flux'][j], flux_f['F140W_fluxerr'][j], 
                flux_f['F160W_flux'][j], flux_f['F160W_fluxerr'][j] ))
            flux_sel_f.close()

19
5304 2 bands_to_exclude
----------------
         'F140W'
7
21258 4 bands_to_exclude
----------------
         'F140W'
11
17482 2 bands_to_exclude
----------------
          'None'


In [8]:
# instantiating flux profile file
flux_sel_f = open(flux_fname, 'w')
flux_sel_f.write( 'ID field z F435W_flux F435W_fluxerr F606W_flux F606W_fluxerr F775W_flux F775W_fluxerr F850LP_flux F850LP_fluxerr F125W_flux F125W_fluxerr F140W_flux F140W_fluxerr F160W_flux F160W_fluxerr\n' )
flux_sel_f.close()

for i in range( len( selection_f ) ):
    gal_id, field = selection_f['ID'][i], selection_f['field'][i]
    category = selection_f['category'][i]
    band = selection_f['band'][i]
        
    try:
        #flux_f = ascii.read('/Users/vega/proposals/hst_cycle_29/profiles/%s_%d_adaptive_annuli_bin_size_%d_sum_multiple_band_snr_cut_2_band_flux_profiles.txt' % (field, gal_id, bin_size) ) #+ '_sigma_4.0_mean_clip_all_pix_flux_profiles.txt')
        #flux_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/profiles_extent/%s_%d_adaptive_annuli_bin_size_%d_sum_multiple_band_snr_cut_2_band_flux_profiles.txt' % (field, gal_id, bin_size) ) #+ '_sigma_4.0_mean_clip_all_pix_flux_profiles.txt')
        flux_f = ascii.read('/Volumes/BEAGLE1/CANDELS/radial_flux_profiles/%s_%d_adaptive_annuli_bin_size_%d_sum_multiple_band_snr_cut_2_band_flux_profiles.txt' % (field, gal_id, bin_size) )
    except:
        print( 'Radial flux profiles have not been produced for %s %d' % (field, gal_id) )
        continue
        
    z = flux_f['z'][0]
    
    if category == 3:
        # compute radius limits
        snr_lims_5 = snr_limits( flux_f, min_snr=5. )
    
        # using one WFC3 band to determine the radius limit as a function of redshift
        if z >= 0.8: 
            snr_lim_5 = snr_lims_5[6] # F160W
        else:
            snr_lim_5 = snr_lims_5[4] # F125W
    
    elif category == 2:
        sb_profile = -2.5 * np.log10( flux_f['%s_flux' % band] / ( flux_f['area_pix'] * 0.06 ** 2) ) + 8.9
    
        # find relative maxima 
        rel_max = argrelextrema( sb_profile, np.greater )
        
        snr_lim_5 = rel_max[0][0]
        
    for j in range( snr_lim_5 + 1 ):
        
        flux_sel_f = open(flux_fname, 'a+')
        flux_sel_f.write('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16}\n'.format(
                field + '_' + str(gal_id) + '_' + str(j), field, flux_f['z'][j], 
                flux_f['F435W_flux'][j], flux_f['F435W_fluxerr'][j], 
                flux_f['F606W_flux'][j], flux_f['F606W_fluxerr'][j],
                flux_f['F775W_flux'][j], flux_f['F775W_fluxerr'][j], 
                flux_f['F850LP_flux'][j], flux_f['F850LP_fluxerr'][j],
                flux_f['F125W_flux'][j], flux_f['F125W_fluxerr'][j], 
                flux_f['F140W_flux'][j], flux_f['F140W_fluxerr'][j], 
                flux_f['F160W_flux'][j], flux_f['F160W_fluxerr'][j] ))
        flux_sel_f.close()

/Users/vega/anaconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log10


### Prepare FITS files for SED fitting with BEAGLE: defining create_fits()
To fit radial flux profiles with BEAGLE, the function create_fits() below will be used. create_fits() reads in ASCII flux profile files and produces a FITS file to be read by BEAGLE. create_fits() requires a name of the ASCII flux profile file (obs\_f\_name), a name of the FITS file (name) and the user must specify a boolean variable len\_, which, if True, will append the length of the FITS file to the end of the file name. 

In [14]:
beagle_path = '/Volumes/Vega/BEAGLE'

def create_fits(obs_f_name, name, len_=True):
    
    # instantiating a FITS file
    prihdr = fits.Header()
    prihdu = fits.PrimaryHDU(header=prihdr)    

    # ASCII radial flux profile file
    obs_f = ascii.read( obs_f_name )
    
    # arrays of galaxy radial bin IDs, HST fluxes and HST flux errors
    id_list, redshift_list, flux_435, fluxerr_435, flux_606, fluxerr_606, flux_775, fluxerr_775, flux_814, fluxerr_814 = [], [], [], [], [], [], [], [], [], []
    flux_850, fluxerr_850, flux_125, fluxerr_125, flux_140, fluxerr_140, flux_160, fluxerr_160 = [], [], [], [], [], [], [], []
    
    # reading in ASCII radial flux profile file and populating arrays
    for i in range( len(obs_f) ):
    
        id_list.append( obs_f['ID'][i] ), redshift_list.append( obs_f['z'][i] )
        
        field_ = obs_f['field'][i]
        
        # correcting for foreground Galactic extinction in each HST band, depending on the field
        if field_ == 'goodsn':
            f435_ext = 10 ** (0.4 * 0.0419)
            f606_ext = 10 ** (0.4 * 0.0276)
            f775_ext = 10 ** (0.4 * 0.0183)
            f814_ext = 10 ** (0.4 * 0.0168)
            f850_ext = 10 ** (0.4 * 0.0138)
            f125_ext = 10 ** (0.4 * 0.0081)
            f140_ext = 10 ** (0.4 * 0.0070)
            f160_ext = 10 ** (0.4 * 0.0063)
        elif field_ == 'goodss':
            f435_ext = 10 ** (0.4 * 0.0288)
            f606_ext = 10 ** (0.4 * 0.0190)
            f775_ext = 10 ** (0.4 * 0.0125)
            f814_ext = 10 ** (0.4 * 0.0115)
            f850_ext = 10 ** (0.4 * 0.0094)
            f125_ext = 10 ** (0.4 * 0.0055)
            f140_ext = 10 ** (0.4 * 0.0049)
            f160_ext = 10 ** (0.4 * 0.0043)
        
        # populating arrays of HST fluxes and flux errors
        # f435w
        if obs_f['F435W_flux'][i] <= 0:
            flux_435.append( -99. )
            fluxerr_435.append( -99. )
        else:
            flux_435.append( obs_f['F435W_flux'][i] * f435_ext )
            fluxerr_435.append( obs_f['F435W_fluxerr'][i]  )
            
        # f606w
        if obs_f['F606W_flux'][i] <= 0:
            flux_606.append( -99. )
            fluxerr_606.append( -99. )
        else:
            flux_606.append( obs_f['F606W_flux'][i] * f606_ext )
            fluxerr_606.append( obs_f['F606W_fluxerr'][i]  )
        
        # f775w
        if obs_f['F775W_flux'][i] <= 0:
            flux_775.append( -99. )
            fluxerr_775.append( -99. )
        else:
            flux_775.append( obs_f['F775W_flux'][i] * f775_ext )
            fluxerr_775.append( obs_f['F775W_fluxerr'][i]  )
            
        # f850lp
        if obs_f['F850LP_flux'][i] <= 0:
            flux_850.append( -99. )
            fluxerr_850.append( -99. )
        else:
            flux_850.append( obs_f['F850LP_flux'][i] * f850_ext )
            fluxerr_850.append( obs_f['F850LP_fluxerr'][i]  )
            
        # f125w
        if obs_f['F125W_flux'][i] <= 0:
            flux_125.append( -99. )
            fluxerr_125.append( -99. )
        else:
            flux_125.append( obs_f['F125W_flux'][i] * f125_ext )
            fluxerr_125.append( obs_f['F125W_fluxerr'][i]  )
            
        # f140w
        if obs_f['F140W_flux'][i] <= 0:
            flux_140.append( -99. )
            fluxerr_140.append( -99. )
        else:
            flux_140.append( obs_f['F140W_flux'][i] * f140_ext )
            fluxerr_140.append( obs_f['F140W_fluxerr'][i]  )
            
        # f160w
        if obs_f['F160W_flux'][i] <= 0:
            flux_160.append( -99. )
            fluxerr_160.append( -99. )
        else:
            flux_160.append( obs_f['F160W_flux'][i] * f160_ext )
            fluxerr_160.append( obs_f['F160W_fluxerr'][i]  )
    
    # table that will be appended to FITS HDU
    tbhdu = fits.BinTableHDU.from_columns(
        [fits.Column(name='ID', format='20A', array=id_list),
        fits.Column(name='z', format='D', array=redshift_list),
        fits.Column(name='flux_435', format='1E', array=flux_435),
        fits.Column(name='fluxerr_435', format='1E', array=fluxerr_435),
        fits.Column(name='flux_606', format='1E', array=flux_606),
        fits.Column(name='fluxerr_606', format='1E', array=fluxerr_606),
        fits.Column(name='flux_775', format='1E', array=flux_775),
        fits.Column(name='fluxerr_775', format='1E', array=fluxerr_775),
        fits.Column(name='flux_814', format='1E', array=flux_814),
        fits.Column(name='fluxerr_814', format='1E', array=fluxerr_814),
        fits.Column(name='flux_850', format='1E', array=flux_850),
        fits.Column(name='fluxerr_850', format='1E', array=fluxerr_850),
        fits.Column(name='flux_125', format='1E', array=flux_125),
        fits.Column(name='fluxerr_125', format='1E', array=fluxerr_125),
        fits.Column(name='flux_140', format='1E', array=flux_140),
        fits.Column(name='fluxerr_140', format='1E', array=fluxerr_140),
        fits.Column(name='flux_160', format='1E', array=flux_160),
        fits.Column(name='fluxerr_160', format='1E', array=fluxerr_160)])

    thdulist = fits.HDUList([prihdu, tbhdu])
    thdulist.writeto(beagle_path + '/data/' + name + '_' + str( len( obs_f ) ) + '.fits', overwrite=True)
    

### Prepare FITS file for SED fitting with BEAGLE
In the cell below, the user uses create_fits() and indicates which ASCII radial flux profile file they wish to convert to a FITS file, the file name of the new FITS file, and whether the length of the file will be appended to the FITS file name. 

In [15]:
#create_fits(obs_f_name='/Users/vega/CANDELS/flux_files/goods_hmag_25_mass_10.0_10.4_z_1.0_1.5_bin_2_part1.txt', 
#                              name='goods_hmag_25_mass_10.0_10.4_z_1.0_1.5_bin_2_part1')

create_fits(obs_f_name='/Volumes/Vega/CANDELS/sed_assumptions_paper/flux_files/remaining_final_cuts_no_vis_fluxes_nov23.txt', 
           name='remaining_final_cuts_no_vis_nov_23_no_upper_limits')


### Create FITS file including upper limits

In [18]:
beagle_path = '/Volumes/Vega/BEAGLE'

def create_fits_upper_limits(obs_f_name, name, snr_lim=2, len_=True):
    
    # instantiating a FITS file
    prihdr = fits.Header()
    prihdu = fits.PrimaryHDU(header=prihdr)    

    # ASCII radial flux profile file
    obs_f = ascii.read( obs_f_name )
    
    # arrays of galaxy radial bin IDs, HST fluxes and HST flux errors
    id_list, redshift_list, flux_435, fluxerr_435, flux_606, fluxerr_606, flux_775, fluxerr_775 = [], [], [], [], [], [], [], []
    flux_850, fluxerr_850, flux_125, fluxerr_125, flux_140, fluxerr_140, flux_160, fluxerr_160 = [], [], [], [], [], [], [], []
    
    # reading in ASCII radial flux profile file and populating arrays
    for i in range( len(obs_f) ):
        
        #id_list.append( '%s_%d_%d' % (field, gal_id, i) ), redshift_list.append( obs_f['z'][i] )
        id_list.append( obs_f['ID'][i] ), redshift_list.append( obs_f['z'][i] )
        
        field_ = obs_f['field'][i]
        
        # correcting for foreground Galactic extinction in each HST band, depending on the field
        if field_ == 'goodsn':
            f435_ext = 10 ** (0.4 * 0.0419)
            f606_ext = 10 ** (0.4 * 0.0276)
            f775_ext = 10 ** (0.4 * 0.0183)
            f814_ext = 10 ** (0.4 * 0.0168)
            f850_ext = 10 ** (0.4 * 0.0138)
            f125_ext = 10 ** (0.4 * 0.0081)
            f140_ext = 10 ** (0.4 * 0.0070)
            f160_ext = 10 ** (0.4 * 0.0063)
        elif field_ == 'goodss':
            f435_ext = 10 ** (0.4 * 0.0288)
            f606_ext = 10 ** (0.4 * 0.0190)
            f775_ext = 10 ** (0.4 * 0.0125)
            f814_ext = 10 ** (0.4 * 0.0115)
            f850_ext = 10 ** (0.4 * 0.0094)
            f125_ext = 10 ** (0.4 * 0.0055)
            f140_ext = 10 ** (0.4 * 0.0049)
            f160_ext = 10 ** (0.4 * 0.0043)
        
        # populating arrays of HST fluxes and flux errors
        # f435w
        if obs_f['F435W_flux'][i] <= 0:
            flux_435.append( -99. )
            fluxerr_435.append( -99. )
        elif obs_f['F435W_flux'][i] / obs_f['F435W_fluxerr'][i] <= snr_lim:
            flux_435.append( 1e-20 )
            fluxerr_435.append( obs_f['F435W_flux'][i] )
        elif np.isnan( obs_f['F435W_fluxerr'][i] ):
            flux_435.append( -99. )
            fluxerr_435.append( -99. )
        else:
            flux_435.append( obs_f['F435W_flux'][i] * f435_ext )
            fluxerr_435.append( obs_f['F435W_fluxerr'][i]  )
            
        # f606w
        if obs_f['F606W_flux'][i] <= 0:
            flux_606.append( -99. )
            fluxerr_606.append( -99. )
        elif obs_f['F606W_flux'][i] / obs_f['F606W_fluxerr'][i] <= snr_lim:
            flux_606.append( 1e-20 )
            fluxerr_606.append( obs_f['F606W_flux'][i] )
        elif np.isnan( obs_f['F606W_fluxerr'][i] ):
            flux_606.append( -99. )
            fluxerr_606.append( -99. )
        else:
            flux_606.append( obs_f['F606W_flux'][i] * f606_ext )
            fluxerr_606.append( obs_f['F606W_fluxerr'][i]  )
        
        # f775w
        if obs_f['F775W_flux'][i] <= 0:
            flux_775.append( -99. )
            fluxerr_775.append( -99. )
        elif obs_f['F775W_flux'][i] / obs_f['F775W_fluxerr'][i] <= snr_lim:
            flux_775.append( 1e-20 )
            fluxerr_775.append( obs_f['F775W_flux'][i] )
        elif np.isnan( obs_f['F775W_fluxerr'][i] ):
            flux_775.append( -99. )
            fluxerr_775.append( -99. )
        else:
            flux_775.append( obs_f['F775W_flux'][i] * f775_ext )
            fluxerr_775.append( obs_f['F775W_fluxerr'][i]  )
            
        # f850lp
        if obs_f['F850LP_flux'][i] <= 0:
            flux_850.append( -99. )
            fluxerr_850.append( -99. )
        elif obs_f['F850LP_flux'][i] / obs_f['F850LP_fluxerr'][i] <= snr_lim:
            flux_850.append( 1e-20 )
            fluxerr_850.append( obs_f['F850LP_flux'][i] )
        elif np.isnan( obs_f['F850LP_fluxerr'][i] ):
            flux_850.append( -99. )
            fluxerr_850.append( -99. )
        else:
            flux_850.append( obs_f['F850LP_flux'][i] * f850_ext )
            fluxerr_850.append( obs_f['F850LP_fluxerr'][i]  )
            
        # f125w
        if obs_f['F125W_flux'][i] <= 0:
            flux_125.append( -99. )
            fluxerr_125.append( -99. )
        elif obs_f['F125W_flux'][i] / obs_f['F125W_fluxerr'][i] <= snr_lim:
            flux_125.append( 1e-20 )
            fluxerr_125.append( obs_f['F125W_flux'][i] )
        elif np.isnan( obs_f['F125W_fluxerr'][i] ):
            flux_125.append( -99. )
            fluxerr_125.append( -99. )
        else:
            flux_125.append( obs_f['F125W_flux'][i] * f125_ext )
            fluxerr_125.append( obs_f['F125W_fluxerr'][i]  )
            
        # f140w
        if obs_f['F140W_flux'][i] <= 0:
            flux_140.append( -99. )
            fluxerr_140.append( -99. )
        elif obs_f['F140W_flux'][i] / obs_f['F140W_fluxerr'][i] <= snr_lim:
            flux_140.append( 1e-20 )
            fluxerr_140.append( obs_f['F140W_flux'][i] )
        elif np.isnan( obs_f['F140W_fluxerr'][i] ):
            flux_140.append( -99. )
            fluxerr_140.append( -99. )
        else:
            flux_140.append( obs_f['F140W_flux'][i] * f140_ext )
            fluxerr_140.append( obs_f['F140W_fluxerr'][i]  )
            
        # f160w
        if obs_f['F160W_flux'][i] <= 0:
            flux_160.append( -99. )
            fluxerr_160.append( -99. )
        elif obs_f['F160W_flux'][i] / obs_f['F160W_fluxerr'][i] <= snr_lim:
            flux_160.append( 1e-20 )
            fluxerr_160.append( obs_f['F160W_flux'][i] )
        elif np.isnan( obs_f['F160W_fluxerr'][i] ):
            flux_160.append( -99. )
            fluxerr_160.append( -99. )
        else:
            flux_160.append( obs_f['F160W_flux'][i] * f160_ext )
            fluxerr_160.append( obs_f['F160W_fluxerr'][i]  )
    
    # table that will be appended to FITS HDU
    tbhdu = fits.BinTableHDU.from_columns(
        [fits.Column(name='ID', format='20A', array=id_list),
        fits.Column(name='z', format='D', array=redshift_list),
        fits.Column(name='flux_435', format='1E', array=flux_435),
        fits.Column(name='fluxerr_435', format='1E', array=fluxerr_435),
        fits.Column(name='flux_606', format='1E', array=flux_606),
        fits.Column(name='fluxerr_606', format='1E', array=fluxerr_606),
        fits.Column(name='flux_775', format='1E', array=flux_775),
        fits.Column(name='fluxerr_775', format='1E', array=fluxerr_775),
        fits.Column(name='flux_850', format='1E', array=flux_850),
        fits.Column(name='fluxerr_850', format='1E', array=fluxerr_850),
        fits.Column(name='flux_125', format='1E', array=flux_125),
        fits.Column(name='fluxerr_125', format='1E', array=fluxerr_125),
        fits.Column(name='flux_140', format='1E', array=flux_140),
        fits.Column(name='fluxerr_140', format='1E', array=fluxerr_140),
        fits.Column(name='flux_160', format='1E', array=flux_160),
        fits.Column(name='fluxerr_160', format='1E', array=fluxerr_160)])

    thdulist = fits.HDUList([prihdu, tbhdu])
    thdulist.writeto(beagle_path + '/data/' + name + '_' + str( len( obs_f ) ) + '.fits', overwrite=True)
    

In [20]:
#create_fits_upper_limits(obs_f_name='/Users/vega/CANDELS/sed_assumptions_paper/flux_files/remaining_final_cuts_no_vis_fluxes.txt', 
#                        name='remaining_final_cuts_no_vis_upper_limits_snr_lim_2')

create_fits_upper_limits(obs_f_name='/Volumes/Vega/CANDELS/sed_assumptions_paper/flux_files/remaining_final_cuts_no_vis_fluxes_nov23.txt', 
                        name='remaining_final_cuts_no_vis_nov_23_upper_limits_snr_lim_2')
